In [9]:
import json, boto3
import pandas as pd
import datetime

In [10]:
# S3 client 생성에 필요한 보안 자격 증명 정보
with open("./API_KEYS.json", "r") as f:
    key = json.load(f)

with open("./DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)
# S3 버킷 정보
pull_bucket_name = 'crawl-data-lake'
push_bucket_name = 'merged-data-storage'
target_folder_prefix = 'wanted/data/'

In [11]:
# S3 섹션 및 client 생성
session = boto3.Session(
    aws_access_key_id=key['aws_access_key_id'],
    aws_secret_access_key=key['aws_secret_key'],
    region_name=key['region']
)

s3 = session.client('s3')

In [12]:
# S3 버킷 정보
pull_bucket_name = bucket_info['pull_bucket_name']
push_bucket_name = bucket_info['push_bucket_name']
target_folder_prefix = bucket_info['target_folder_prefix']

In [13]:
# 특정 폴더 내 파일 목록 가져오기
response = s3.list_objects_v2(Bucket=pull_bucket_name, Prefix=target_folder_prefix, Delimiter='/')
#curr_date = datetime.datetime.now()
curr_date = datetime.date(2024, 8, 19)

# curr_date 보다 날짜가 늦은 data josn 파일 metadata 객체 분류
if 'Contents' in response:
    target_file_list = [obj for obj in response['Contents'] if curr_date <= obj['LastModified'].date()]
else:
    print("No objects found in the folder.")

In [16]:
# 분류한 list 기준으로 data 읽어오기
for obj in target_file_list:
    response = s3.get_object(Bucket=pull_bucket_name, Key=obj['Key'])
    json_context = response['Body'].read().decode('utf-8')
    json_data = [json.loads(line) for line in json_context.splitlines()]
    
    df = pd.DataFrame(json_data)
    break